In [1]:
!pip install transformers datasets peft trl bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 200.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 112.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 189.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 kB 144.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 146.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 189.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 170.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!pip install --upgrade bitsandbytes


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
from datasets import Dataset  # or load_dataset if from file
from transformers import AutoTokenizer
import json

In [2]:
from transformers import  AutoModelForCausalLM, BitsAndBytesConfig, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType
from peft.utils.other import prepare_model_for_kbit_training
from datasets import load_dataset
import torch

In [3]:
def format_example(example):
    system_prompt = "You are a tutor. Give responses as a tutor. Do not give full code solutions — only partial solutions, hints, or small examples.\n\n"
    user_input = example['prompt'].strip()
    response = example['response'].strip()

    full_prompt = (
        "<|system|>\n"
        f"{system_prompt.strip()}\n"
        "<|user|>\n"
        f"{user_input.strip()}\n"
        "<|assistant|>\n"
        f"{response.strip()}"
    )

    tokenized = tokenizer(
        full_prompt,
        padding="max_length",
        truncation=True,
        max_length=1024,
        return_tensors="pt"
    )
    tokenized = {k: v.squeeze(0) for k, v in tokenized.items()}

    # Mask all tokens before assistant's answer
    assistant_start = full_prompt.find("<|assistant|>")
    assistant_token_ids = tokenizer(full_prompt[:assistant_start], return_tensors="pt").input_ids.squeeze(0)
    labels = tokenized["input_ids"].clone()
    labels[:len(assistant_token_ids)] = -100
    tokenized["labels"] = labels

    return tokenized

In [11]:
!rm -rf ~/.cache/huggingface/hub/models--bigcode--starcoder2-3b


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoder2-3b", trust_remote_code=True)
bnb_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/12.1G [00:00<?, ?B/s]

In [20]:
def format_example(example):
    prompt = example["prompt"].strip()
    response = example["response"].strip()

    full_text = prompt + "\n" + response + tokenizer.eos_token

    # Tokenize full sequence
    tokenized = tokenizer(full_text, truncation=True, max_length=1024, padding="max_length")
    
    # Mask prompt (so only response is used for training loss)
    prompt_token_count = len(tokenizer(prompt, truncation=True, max_length=1024)["input_ids"])
    labels = tokenized["input_ids"].copy()
    labels[:prompt_token_count] = [-100] * prompt_token_count
    tokenized["labels"] = labels

    return tokenized


In [21]:
# Load your dataset from JSON
with open("./merged_output.json", "r") as f:
    raw_data = json.load(f)

dataset = Dataset.from_list(raw_data)

# Load tokenizer with proper config for Starcoder2

# Ensure tokenizer has a pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    
tokenized_dataset = dataset.map(
    format_example,
    remove_columns=dataset.column_names,
    batched=False,
    desc="Tokenizing dataset...",
)
    
    
    


Tokenizing dataset...:   0%|          | 0/102000 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset.save_to_disk("tokenized_starcoder2_dataset")


In [22]:
from trl import SFTTrainer, SFTConfig


In [23]:


# ---- 2. Load Base Model with 8-bit Quantization ----


model = prepare_model_for_kbit_training(model)

# ---- 3. Apply LoRA using PEFT ----
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj", "v_proj", "k_proj", "o_proj",
        "up_proj", "down_proj", "gate_proj"
    ],
    
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


trainable params: 9,093,120 || all params: 3,039,464,448 || trainable%: 0.2992


In [26]:
sft_config = SFTConfig(
    output_dir="./checkpoints",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    auto_find_batch_size=False,  # Set to True if you get OOM error
    num_train_epochs=2,
    logging_steps=200,
    save_steps=500,
    save_total_limit=2,
    fp16=False,
    bf16=True,
    learning_rate=3e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    optim="adamw_torch_fused",  # Use fused optimizer for faster training
    max_grad_norm=1.0,
    gradient_checkpointing=True,  # Saves memory at slight compute cost
    report_to="none",
    
)
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=sft_config,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)

Truncating train dataset:   0%|          | 0/102000 [00:00<?, ? examples/s]

In [17]:
import torch
import numpy as np

torch.serialization.add_safe_globals([np.core.multiarray._reconstruct])


/tmp/ipykernel_164/1360165729.py:4: DeprecationWarning: numpy.core is deprecated and has been renamed to numpy._core. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.multiarray.
  torch.serialization.add_safe_globals([np.core.multiarray._reconstruct])


In [18]:
_original_torch_load = torch.load

def patched_load(*args, **kwargs):
    kwargs["weights_only"] = False  # Force loading full state
    return _original_torch_load(*args, **kwargs)

torch.load = patched_load


In [ ]:

trainer.train()



/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
200,3.354700
